In [1]:
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib import rnn
from sklearn.model_selection import train_test_split
import numpy as np

In [2]:
###load data first
features = np.loadtxt("features.txt", delimiter=",")
labels = np.loadtxt("labels.txt", delimiter=",")
x_all = features
#y_all = labels
y_all = np.array([labels, -(labels-1)]).T 
# normalize
x_all = (x_all - x_all.min(0)) / x_all.ptp(0)
#Split the data in train & test
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all)

In [3]:
# Training Parameters

learning_rate = 0.01
training_steps = 1800
batch_size = 128
display_step = 200

# Network Parameters
num_input = 61 # MNIST data input (img shape: 28*28)
timesteps = 4 # timesteps
num_hidden = 61 # hidden layer num of features
num_classes = 2 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, timesteps, num_input])
Y = tf.placeholder("float", [None, num_classes])
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([num_hidden, num_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([num_classes]))
}

def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, timesteps, 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(num_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

logits = RNN(X, weights, biases)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [8]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    # total_batch = int(len(X_train)/batch_size)
    # X_batches = np.array_split(x_train, total_batch)
    # Y_batches = np.array_split(y_train, total_batch)
    
    for step in range(1, training_steps+1):
        # batch_x, batch_y = mnist.train.next_batch(batch_size)
        rand_index = np.random.choice(len(x_train), size=batch_size)
        batch_x = x_train[rand_index]
        batch_y = y_train[rand_index]
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        #print(batch_x,batch_y)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    #test_len = 128
    # test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    x_test = x_test.reshape((-1, timesteps, num_input))
    # test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: x_test, Y: y_test}))

Step 1, Minibatch Loss= 0.7226, Training Accuracy= 0.492
Step 200, Minibatch Loss= 0.6619, Training Accuracy= 0.648
Step 400, Minibatch Loss= 0.6033, Training Accuracy= 0.812
Step 600, Minibatch Loss= 0.5152, Training Accuracy= 0.742
Step 800, Minibatch Loss= 0.5829, Training Accuracy= 0.836
Step 1000, Minibatch Loss= 0.4824, Training Accuracy= 0.969
Step 1200, Minibatch Loss= 0.5217, Training Accuracy= 0.656
Step 1400, Minibatch Loss= 0.4336, Training Accuracy= 0.930
Step 1600, Minibatch Loss= 0.3309, Training Accuracy= 0.992
Step 1800, Minibatch Loss= 0.2475, Training Accuracy= 0.930
Optimization Finished!
Testing Accuracy: 0.90802675
